## Essai API ADEME AVEC PYTHON

In [2]:
import requests as req
import json
import pandas as pandas
import numpy as np

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

# Fichier des codes postaux 
df_cp=pandas.read_csv("data/adresses-69.csv",sep=";")
cp69=df_cp["code_postal"].unique()
print("Nombres de cp",len(cp69))
print(cp69)


Nombres de cp 91
[69790 69170 69250 69380 69009 69008 69006 69007 69005 69001 69004 69002
 69003 69780 69360 69124 69580 69720 69960 69320 69800 69140 69330 69970
 69730 69740 69150 69680 69510 69390 69910 69100 69640 69770 69400 69430
 69820 69460 69200 69120 69420 69670 69890 69620 69240 69160 69220 69440
 69490 69700 69560 69590 69270 69870 69210 69850 69930 69690 69550 69650
 69830 69290 69230 69610 69110 69190 69450 69370 69280 69470 69480 69530
 69310 69600 69350 69860 69760 69840 69540 69520 69340 69130 69570 69660
 69115 69260 69410 69630 69300 69500 69126]


C:\Users\ychen\AppData\Local\Temp\ipykernel_37648\2248590103.py:10: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cp=pandas.read_csv("data/adresses-69.csv",sep=";")


In [ ]:
import os, time
import requests
import pandas as pd
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# ---------- Paramètres ----------
BASE_URL = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"
YEARS = range(2020, 2025)                # 2020 → 2024
PAGE_SIZE = 1000                         # 1000 conseillés pour réduire erreurs réseau
OUT_CSV = "donnees_dpe_rhone.csv"

# ---------- Session HTTP avec retries ----------
session = requests.Session()
retries = Retry(
    total=5,                # nb tentatives
    connect=3, read=3,
    backoff_factor=0.5,     # délai exponentiel: 0.5, 1, 2, ...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("https://", HTTPAdapter(max_retries=retries))

# ---------- Chargement des CP (sans warning) ----------
# force le type pour code_postal et low_memory=False pour éviter DtypeWarning
df_cp = pd.read_csv("adresses-69.csv", sep=";", dtype={"code_postal": str}, low_memory=False)
cp69 = df_cp["code_postal"].dropna().unique().tolist()
print("Nombre de CP:", len(cp69))

# ---------- Utilitaire: fetch d'une page ----------
def fetch_page(cp: str, year: int, offset: int):
    borne1 = f"{year}-01-01"
    borne2 = f"{year}-12-31"
    params = {
        "size": PAGE_SIZE,
        "from": offset,  # pagination par offset
        "q": cp,
        "q_fields": "code_postal_ban",
        "qs": f"date_reception_dpe:{borne1} TO {borne2}"
        # Pas de 'select' => toutes les colonnes
    }
    # stream=False (par défaut) limite les erreurs chunked
    r = session.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    js = r.json()
    results = js.get("results", [])
    total = js.get("total", 0)
    return results, total

# ---------- Ecriture CSV en append ----------
def append_to_csv(df: pd.DataFrame, path: str, header_manager: dict):
    # On fige le schéma la 1ère fois: toutes les colonnes de la première page
    if header_manager.get("columns") is None:
        header_manager["columns"] = list(df.columns)
        df = df.reindex(columns=header_manager["columns"])
        df.to_csv(path, index=False, mode="w", header=True)
        header_manager["written_header"] = True
    else:
        # on aligne les colonnes sur le schéma figé
        df = df.reindex(columns=header_manager["columns"])
        df.to_csv(path, index=False, mode="a", header=False)

# ---------- Boucle principale ----------
header_manager = {"columns": None, "written_header": False}
total_rows_written = 0

for cp in cp69:
    for year in YEARS:
        print(f"\n=== CP {cp} — Année {year} ===")
        offset = 0
        page_idx = 1
        total_for_query = None

        while True:
            try:
                results, total = fetch_page(cp, year, offset)
            except requests.exceptions.RequestException as e:
                print(f"[WARN] Requête échouée (offset {offset}). Nouvelle tentative...")
                time.sleep(1.5)
                # une nouvelle tentative:
                try:
                    results, total = fetch_page(cp, year, offset)
                except Exception as e2:
                    print(f"[ERROR] Abandon de la page offset={offset}: {e2}")
                    break  # on sort de la boucle de cette année/CP

            if total_for_query is None:
                total_for_query = total
                print(f"Total annoncé par l'API pour CP {cp} / {year}: {total_for_query}")

            if not results:
                # plus de données
                break

            df_page = pd.DataFrame(results)

            # écriture incrémentale
            append_to_csv(df_page, OUT_CSV, header_manager)
            total_rows_written += len(df_page)

            print(f"Page {page_idx} | lignes écrites: {len(df_page)} | total cumulé: {total_rows_written}")

            # pagination
            offset += PAGE_SIZE
            page_idx += 1

            # micro pause pour éviter le throttling
            time.sleep(0.2)

print(f"\n✅ Terminé. Fichier écrit: {OUT_CSV} | Lignes totales: {total_rows_written}")
print("NB: Le schéma CSV est figé sur la 1ère page rencontrée (colonnes identiques ensuite).")